In [1]:
sealed trait Actions {
    def doIt(): Actions
    def isCallable: Boolean
}
case class Call( thunk: () => Actions) extends Actions {
    def doIt(): Actions =  {thunk()} // Thing that executes the thunk
    def isCallable: Boolean = true 
}

case class Done[T](v: T) extends Actions { // Termination action
    def doIt(): Actions = throw new IllegalArgumentException("Cannot ask me to do it when I am already done")
    def isCallable: Boolean = false 
}

defined trait Actions
defined class Call
defined class Done

In [2]:
def factorial_cps[T](x: Int, k: Int => T): T = {
    if (x <= 1)
        k(1) // Call cont. on base case
    else 
        factorial_cps( x - 1, v => k(x * v)) // factorial call with cont.
}

defined function factorial_cps

In [7]:
def factorial_trampoline(x: Int, k: Int => Actions): Actions = {
    println("In Factorial_Trampoline Function")
    if (x <= 1)
        Call (() => k(1)) // Thunk up k(1)
    else
        Call (() => factorial_trampoline(x-1, (v)=> { Call( () => k(x * v)) } )) // Thunk up all inner fn. calls as well.
}

defined function factorial_trampoline

In [8]:
def factorial_main(x: Int): Int = {
    val k0 = (v: Int) => {Done(v)} // Terminal Continuation
    var currentThunk = factorial_trampoline(x, k0)
    while (currentThunk.isCallable) {
        println("In Factorial Main Function")
        currentThunk = currentThunk.doIt()
    }
    currentThunk match {
        case Done(v: Int) => v
        case _ => throw new AssertionError("This is impossible since when the trampoline loop is done, we should be done.")
    }
    
}

defined function factorial_main

In [9]:
factorial_main(10)

In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial_Trampoline Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function
In Factorial Main Function


res8: Int = 3628800

In [11]:
def fibonacci_cps[T](x: Int, k : Int => T): T = {
    if (x <= 2)
        k(1)
    else
        fibonacci_cps( x -1 , v1 => { fibonacci_cps(x -2, v2 => {k (v1 + v2)})})
} // Thing that failed us

defined function fibonacci_cps

In [13]:
def fibonacci_trampoline[T](x: Int, k: Int => Actions): Actions = {
    if (x <= 2)
        Call ( () => k(1)) // Wrap up the call into a thunk
    else 
        Call ( () => {
            fibonacci_trampoline( x-1, 
                         v1 => {
                             Call( () => fibonacci_trampoline(x-2, v2 => {
                                     Call (() => k(v1+v2))
                                 }) 
                             )
                         }
                         )
        }  ) // Wrap up every tail call into a thunk
}

defined function fibonacci_trampoline

In [14]:
def trampolineIt(thunkedCPSStyleFun: (Int, Int => Actions) => Actions, x0: Int): Int = {
    val k0 = ( v: Int ) => Done(v) // Create a terminal continuation
    var k = thunkedCPSStyleFun(x0, k0)
    while (k.isCallable){
        k = k.doIt()
    }
    k match {
        case Done(v: Int)=> v
        // This case below should not be reachable.
        case _ => throw new AssertionError("This is impossible since when the trampoline loop is done, we should be done.")
    }
}

defined function trampolineIt

In [15]:
def fibonacci(x: Int) = trampolineIt(fibonacci_trampoline, x)

defined function fibonacci

In [16]:
fibonacci(10)

res15: Int = 55

In [17]:
fibonacci(20)

res16: Int = 6765

In [18]:
fibonacci(40)

res17: Int = 102334155

In [19]:
import scala.util.control.TailCalls.{ TailRec, done, tailcall }
// Scala library does have the trampoline

def fibonacci_cps_working(x: Int, k: Int => TailRec[Int]): TailRec[Int] = {
    if (x <= 2)
        tailcall(k(1)) // Call(() => k(1))
    else
            tailcall( fibonacci_cps_working(
                x-1, v1 => (
                        tailcall( fibonacci_cps_working(x-2, 
                                             v2 => { tailcall(k(v1 + v2))}
                                             ) )
                    )
            )
        )
}


val v0 = (fibonacci_cps_working(40, (v:Int) => (done(v)))).result


import scala.util.control.TailCalls.{ TailRec, done, tailcall }
// Scala library does have the trampoline


defined function fibonacci_cps_working
v0: Int = 102334155